# 771948_A23_T3A - Group Work Assignment #
# Task 2 - Multi-label image-based digit classification problem #
## Assignment by Chris Mintz 202369825 and Antonia Agunbiade 202375309 ##

---
### Additional notes from class chat ###
#### 1. Labels from dataset will be folder names
#### 2. Beware of the multi image model classifying into triplets instead of single digits. The output shape should be 1000 instead of 10
#### 3. DO NOT recombine the split data. It has to be used as is as per TA Khadjia
#### 4. From Khadjia: The provided splits into training, validation and testing were designed to simulate real-world scenarios. Teams must develop a baseline model using these splits and gthen improve upon this baseline through various techniques such as preprocessing and model architecture enhancements.
---

# Task 6 - Load and preprocess the dataset for multi-label image-based digit classification task

## <b>References</b>

Johns, Ray. (2024). PyTorch vs TensorFlow for your Python Deep Learning Project. Available at: https://realpython.com/pytorch-vs-tensorflow/. [Accessed Aug 5, 2024]

In [2]:
# For this assignment we have chosen to use the TensorFlow over PyTorch as TensorFlow 2.0 has eager execution and the Keras APIs have more prefabricated components for us to use.
import tensorflow as tf
# TO DO: we might be able to remove this global import
from tensorflow import keras
# there is a lint problem access problem with the latest release of TensorFlow/Keras so we have to go direct to the root call
from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator


# Define our image parameters
image_height = 84  
image_width = 84   
batch_size = 32    

# Data augmentation and preprocessing parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to [0, 1]
    shear_range=0.2,         # Apply shear transformations
    zoom_range=0.2,          # Apply zoom transformations
    horizontal_flip=True)    # Randomly flip images horizontally

In [5]:
# Load the images from a locally mapped source (see README.md for details on setting up local drive mapping)
train_generator = train_datagen.flow_from_directory('z:',
                                                    target_size=(image_height, image_width),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

Found 100000 images belonging to 1 classes.


In [6]:
# TO DO: PICK UP HERE
# Access the data
for images, labels in train_generator:
    break 

ImportError: Could not import PIL.Image. The use of `load_img` requires PIL.

# Task 7 - Build a convultional neural network (CNN) model for the multi-label image-based digit classification task

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Model Architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))  # Sigmoid for multi-label

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

# Data Augmentation (Optional but highly recommended)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'C:\\Users\\aagunbia\\UNI WORK\\Public-1\\module 3 final assignment\\dataset2,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical')  # Categorical for multi-label

validation_generator = test_datagen.flow_from_directory(
        'validation_data_directory',
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical')

# Train the model
model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size)


SyntaxError: invalid decimal literal (2773266005.py, line 10)

# Task 8 - Fine-tune the CNN model using appropriate techniques like hyperparameter tuning, cross-validation, etc

# Task 9 - Visualize the dataset and the CNN model's results where applicable with feature maps, learning curves, etc

# Task 10 - Report the final performance of the CNN model using appropriate performance metrics like accuracy, F1-score, etc

# Task 11 -Decide on the best model for classification and CNN architecture for digital recognition